In [ ]:
import pandas as pd
from github import Github, GithubObject
import time

In [ ]:
g = Github('YOUR_TOKEN')

In [ ]:
issues = pd.read_csv('data/issues.csv')
issues = issues.fillna('')
issues.reset_index()

In [ ]:
milestones = pd.read_csv('data/milestones.csv')
milestones = milestones.fillna('')
milestones

In [ ]:
labels = pd.read_csv('data/labels.csv')
labels = labels.fillna('')
labels

In [ ]:
githubUsers = {
    'LettyLettebowskie': 'Letty',
}

In [ ]:
repo = g.get_repo('letty/gitlab-issue-import')
print(repo)

In [ ]:
issueLookup = {}
githubLabels = repo.get_labels()

for label in githubLabels:
    issueLookup[label.name] = label

for index, row in labels.iterrows():
    if row['name'] not in issueLookup.keys():
        color = row['color'].replace('#', '') if 'color' in labels and len(row['color']) > 1 else 'eeeeee'
        desc = row['description'] if 'description' in labels else ''
        repo.create_label(name=row['name'], color=color, description=desc)

In [ ]:
milestoneLookup = {}
githubMilestones = repo.get_milestones(state='all')

for milestone in githubMilestones:
    milestoneLookup[milestone.title] = milestone

for index, row in milestones.iterrows():
    if row['title'] not in milestoneLookup.keys():
        milestone = repo.create_milestone(title=row['title'], state=row['state'])
        milestoneLookup[row['title']] = milestone

In [ ]:
for index, row in issues.iterrows():
    labels = []
    if len(row['Labels']) > 1:
        labels = row['Labels'].split(',')
    assigneesGitlab = []
    if len(row['Assignee Username']) > 1:
        assigneesGitlab = row['Assignee Username'].split(',')
    assigneesGitlab = row['Assignee Username'].split(',')
    assignees = []
    droppedAssignees = ''
    for aG in assigneesGitlab:
        if aG in githubUsers.keys():
            assignees.append(githubUsers[aG])
        else:
            droppedAssignees += '{0} ,'.format(aG)
    
    closed = True if row['Closed At (UTC)'] else False
    body = ''
    if droppedAssignees != '':
        body += 'Assignees from Gitlab without Github Accounts: {0} \n'.format(droppedAssignees)
    body += row['Description']
    issue = repo.create_issue(
        title=row['Title'],
        body=body,
        milestone=milestoneLookup[row['Milestone']] if row['Milestone'] in milestoneLookup.keys() else GithubObject.NotSet,
        labels=labels,
        assignees=assignees,
    )
    if closed:
        issue.edit(state='closed')
    print('id:{0} closed: {1} - title: {2}'.format(index, closed, row['Title']))
    time.sleep(15)